# **ACTIVIDAD 2b -Mamdani docente-**

1. Un docente le solicita que automatice el proceso de puntuación de su materia. La materia consta de un único examen, puntuado entre 0 y 100, y además una nota de concepto con tres valores (regular, bueno, excelente). Diseñe un FIS Mamdani que permita modificar la nota final (1-10) combinando ambas variables de entrada.

2. Recuerde que la nota de concepto sólo modula la nota del examen. Es decir, la nota del examen tiene más peso en el resultado final que el concepto. No obstante, la nota de concepto debe influir en la puntuación final.

3. Explicite las reglas que ha elegido para esta tarea, y dé ejemplos de valores de entrada y salida. Defina los valores máximos y mínimos posibles para cada variable.

In [78]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Definir las variables de entrada y salida
nota_examen = ctrl.Antecedent(np.arange(0, 101, 1), 'nota_examen')
nota_concepto = ctrl.Antecedent(np.arange(0, 3, 1), 'nota_concepto')
nota_final = ctrl.Consequent(np.arange(0, 11, 0.1), 'nota_final')  # Rango hasta 11

# Definir los conjuntos difusos para cada variable
nota_examen['muy_baja'] = fuzz.trimf(nota_examen.universe, [0, 0, 20])
nota_examen['baja'] = fuzz.trimf(nota_examen.universe, [15, 30, 45])
nota_examen['media_baja'] = fuzz.trimf(nota_examen.universe, [40, 50, 60])
nota_examen['media_alta'] = fuzz.trimf(nota_examen.universe, [55, 70, 85])
nota_examen['alta'] = fuzz.trimf(nota_examen.universe, [75, 85, 95])
nota_examen['muy_alta'] = fuzz.trimf(nota_examen.universe, [90, 100, 100])

nota_concepto['regular'] = fuzz.trimf(nota_concepto.universe,  [0, 0, 1])
nota_concepto['bueno'] = fuzz.trimf(nota_concepto.universe, [0, 1, 2])
nota_concepto['excelente'] = fuzz.trimf(nota_concepto.universe,  [1, 2, 2])

# Redefinimos los conjuntos para la nota final con más valores intermedios
nota_final['muy_baja'] = fuzz.trimf(nota_final.universe, [1, 2, 3])
nota_final['baja'] = fuzz.trimf(nota_final.universe, [3, 4, 5])
nota_final['media_baja'] = fuzz.trimf(nota_final.universe, [5, 6, 7])
nota_final['media'] = fuzz.trimf(nota_final.universe, [6, 7, 8])
nota_final['media_alta'] = fuzz.trimf(nota_final.universe, [7, 8, 9])
nota_final['alta'] = fuzz.trimf(nota_final.universe, [8, 9, 10])
nota_final['excelente'] = fuzz.trimf(nota_final.universe, [9, 10, 11])

# Definir las reglas ajustadas para menor influencia del concepto
reglas = [
    # Nota muy alta
    ctrl.Rule(nota_examen['muy_alta'] & nota_concepto['excelente'], nota_final['excelente']),
    ctrl.Rule(nota_examen['muy_alta'] & nota_concepto['bueno'], nota_final['excelente']),
    ctrl.Rule(nota_examen['muy_alta'] & nota_concepto['regular'], nota_final['alta']),
    # Nota alta
    ctrl.Rule(nota_examen['alta'] & nota_concepto['excelente'], nota_final['alta']),
    ctrl.Rule(nota_examen['alta'] & nota_concepto['bueno'], nota_final['alta']),
    ctrl.Rule(nota_examen['alta'] & nota_concepto['regular'], nota_final['alta']),
    # Nota media alta
    ctrl.Rule(nota_examen['media_alta'] & nota_concepto['excelente'], nota_final['alta']),
    ctrl.Rule(nota_examen['media_alta'] & nota_concepto['bueno'], nota_final['media_alta']),
    ctrl.Rule(nota_examen['media_alta'] & nota_concepto['regular'], nota_final['media']),

    # Nota media baja
    ctrl.Rule(nota_examen['media_baja'] & nota_concepto['excelente'], nota_final['media']),
    ctrl.Rule(nota_examen['media_baja'] & nota_concepto['bueno'], nota_final['media_baja']),
    ctrl.Rule(nota_examen['media_baja'] & nota_concepto['regular'], nota_final['baja']),
    # Nota baja

    ctrl.Rule(nota_examen['baja'] & nota_concepto['excelente'], nota_final['baja']),
    ctrl.Rule(nota_examen['baja'] & nota_concepto['bueno'], nota_final['baja']),
    ctrl.Rule(nota_examen['baja'] & nota_concepto['regular'], nota_final['baja']),
    # Nota muy baja
    ctrl.Rule(nota_examen['muy_baja'] & nota_concepto['excelente'], nota_final['baja']),
    ctrl.Rule(nota_examen['muy_baja'] & nota_concepto['bueno'], nota_final['muy_baja']),
    ctrl.Rule(nota_examen['muy_baja'] & nota_concepto['regular'], nota_final['muy_baja']),
]

# Crear el sistema de control
sistema_control = ctrl.ControlSystem(reglas)
simulador = ctrl.ControlSystemSimulation(sistema_control)

def calcula_nota_final(nota_examen, nota_concepto):
    simulador.input['nota_examen'] = nota_examen
    simulador.input['nota_concepto'] = nota_concepto
    simulador.compute()
    nota_fuzzy = simulador.output['nota_final']
    if (nota_concepto==0):
      return (max(1, min(10, -nota_fuzzy*0.1+nota_examen*.09)))
    else:
      return (max(1, min(10, nota_fuzzy*0.15+nota_examen*.09)))

# Ejemplos de cálculo
print(f"Nota final: {calcula_nota_final(0, 0):.2f}")
print(f"Nota final: {calcula_nota_final(60,2):.2f}")
print(f"Nota final: {calcula_nota_final(20,2):.2f}")


Nota final: 1.00
Nota final: 6.75
Nota final: 2.40
